セグメントから最適化

In [95]:
from pulp import LpProblem, LpVariable, LpMaximize, LpStatus, lpSum, value
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import math
import seaborn as sns
import time

In [96]:
# Read files
files = glob('../*/*coupon/*.csv')
cust_df = pd.read_csv(files[0])
prob_df = pd.read_csv(files[1])
# print(files)
# print(cust_df.shape)
# print(prob_df.shape)
display(cust_df.head(3))
display(prob_df.head(3))

,customer_id,age_cat,freq_cat
0,1,age20~34,freq2
1,2,age35~49,freq0
2,3,age35~49,freq0


,age_cat,freq_cat,segment_id,prob_dm1,prob_dm2,prob_dm3
0,age~19,freq0,1,0.07,0.12,0.29
1,age~19,freq1,2,0.21,0.30,0.58
2,age~19,freq2,3,0.28,0.39,0.74


In [99]:
# クーポン付与による来客率の増加分を作成
copy_df = prob_df.copy()
copy_df.columns = ['age', 'freq', 'seg_id', 'prob_1', 'prob_2', 'prob_3']
for i in range(3):
    copy_df[f'prob_({i+1}-1)'] = copy_df[f'prob_{i+1}'] - copy_df['prob_1']
# copy_df['prob_(2-1)'] = copy_df['prob_2'] - copy_df['prob_1']
# copy_df['prob_(3-1)'] = copy_df['prob_3'] - copy_df['prob_1']
# print(copy_df.shape)
# display(copy_df.head(3))

In [100]:
# ダイレクトメールの種類と人数を追加し結合
p_df = pd.DataFrame()
for i, (prob, diff) in enumerate(zip(['prob_1', 'prob_2', 'prob_3'], ['prob_(1-1)', 'prob_(2-1)', 'prob_(3-1)'])):
    tmp_df = copy_df[['age', 'freq', 'seg_id', prob, diff]].copy()
    tmp_df['dm'] = i+1
    tmp_df['members'] = cust_df.merge(prob_df, on=['age_cat', 'freq_cat']).groupby('segment_id').count()['customer_id'].to_list()
    tmp_df.columns = ['age', 'freq', 'seg_id', 'prob', 'prob_diff', 'dm', 'members']
    p_df = pd.concat([p_df, tmp_df], axis=0)
# display(p_df.head(3))

In [103]:
# コスト列作成
def cost_calculation(x):
    costs = {1: 0, 2: 1000, 3: 2000}
    return costs[x]
p_df['cost'] = p_df['dm'].apply(cost_calculation)

# 変数列作成
p_df['variable'] = [
    LpVariable(f'x_s{s:02}m{m}', cat='Continuous', lowBound=0, upBound=1) for s, m in zip(p_df['seg_id'], p_df['dm'])
    ]

# print(p_df.shape)
# display(p_df[p_df['dm'] == 3].head(3))

In [104]:
# Maxmize
problem = LpProblem(name='DiscountCouponProblem02', sense=LpMaximize)

# 最大化する値
problem += lpSum(p_df['variable'] * p_df['prob_diff'] * p_df['members'])

# 制約条件
for k, v in p_df.groupby('seg_id'):
    problem += lpSum(v['variable'].sum()) == 1

# 制約条件
problem += lpSum(p_df['variable'] * p_df['prob'] * p_df['cost'] * p_df['members']) <= 1000000

# 制約条件
for v in p_df['variable']:
    problem += v >= 0.1

# print(problem)

In [108]:
time_start = time.time()
status = problem.solve()
time_stop = time.time()
p_df['result'] = p_df['variable'].apply(value)

print(f'Status : {LpStatus[status]}')
print(f'Result : {value(problem.objective):.04}')
print(f'Time : {time_stop - time_start:.03} (sec)')

Status : Optimal
Result : 326.1
Time : 0.109 (sec)


In [109]:
# すべてのセグメントにおいて合計 1
for k, v in p_df.groupby('seg_id'):
    print(v['result'].sum(), end=', ')

1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 

In [110]:
# 予算 1,000,000 以下
(p_df['result'] * p_df['prob'] * p_df['cost'] * p_df['members']).sum()

999999.99968

In [111]:
# セグメント別の dm10% 以上
for i, v in enumerate(p_df['result']):
    print(f'{v:.03f}', end=', ')
    if i % 10 == 9:
        print()

0.100, 0.800, 0.800, 0.800, 0.800, 0.800, 0.800, 0.800, 0.100, 0.800, 
0.800, 0.800, 0.100, 0.800, 0.800, 0.800, 0.800, 0.100, 0.100, 0.100, 
0.100, 0.100, 0.100, 0.100, 0.800, 0.100, 0.100, 0.100, 0.278, 0.100, 
0.100, 0.100, 0.100, 0.100, 0.100, 0.100, 0.100, 0.100, 0.100, 0.100, 
0.100, 0.100, 0.100, 0.100, 0.622, 0.100, 0.100, 0.100, 

In [113]:
# 結果
result_df = p_df.pivot_table(index='seg_id', columns='dm', values='result')
result_df.columns = ['send_dm1', 'send_dm2', 'send_dm3']
result_df

,send_dm1,send_dm2,send_dm3
seg_id,,,
1,0.1,0.800000,0.100000
2,0.8,0.100000,0.100000
3,0.8,0.100000,0.100000
4,0.8,0.100000,0.100000
5,0.8,0.100000,0.100000
6,0.8,0.100000,0.100000
7,0.8,0.100000,0.100000
8,0.8,0.100000,0.100000
9,0.1,0.800000,0.100000
